In [104]:
import os
from langchain.chat_models import init_chat_model
from langchain_groq import ChatGroq
from langchain_core.documents import Document



### chat model

In [105]:
Groq_api_key = os.getenv("GROQ_API_KEY")

def load_model(model_name: str, api_key: str):
    # model = ChatGroq(model_name=model_name, api_key=api_key)
    # return model
    model = init_chat_model(model_name=model_name, api_key=api_key)
    return model

llm = load_model(model_name="llama3-8b-8192", api_key=Groq_api_key)
print(llm)

### embedding model

In [106]:
from langchain_huggingface import HuggingFaceEmbeddings
def embedding_model(model_name: str):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    return embeddings

embeddings = embedding_model(model_name="sentence-transformers/all-mpnet-base-v2")


## test embedding model 
text = "this is the document about embedding model"

query_result = embeddings.embed_query(text)
print(query_result[:3])

embeddings




[0.010605454444885254, -0.08976001292467117, -0.014820730313658714]


HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

### vector db

In [107]:
from langchain_chroma import Chroma
# from langchain.schema import Document


def create_vector_db(text_docs: list[str]):
    # docs = [Document(page_content=t) for t in text_docs]

    return Chroma(
        collection_name="rag-collection",
        embedding_function=embeddings,
        persist_directory="chroma_db"
    )

    # vector_store.add_documents(docs)
    # return vector_store


Indexing
1. Load: First we need to load our data. This is done with Document Loaders.
2. Split: Text splitters break large Documents into smaller chunks. This is useful both for indexing data and passing it into a model, as large chunks are harder to search over and won’t fit in a model’s finite context window.
3. Store: We need somewhere to store and index our splits, so that they can be searched over later. This is often done using a VectorStore and Embeddings model.

In [125]:
import bs4
from langchain_community.document_loaders import WebBaseLoader




def load_web_page(url: str):
    loader = WebBaseLoader(url)
    docs = loader.load()
    return docs
    # bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
    # loader = WebBaseLoader(url, bs_kwargs={"parse_only": bs4_strainer})
    # docs = loader.load()
    # return docs

docs = load_web_page("https://github.blog/ai-and-ml/github-copilot/how-to-write-a-great-agents-md-lessons-from-over-2500-repositories/", "www.google.com")
# print(docs[0].page_content[:2000])

print(len(docs))


TypeError: load_web_page() takes 1 positional argument but 2 were given

In [109]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
    
# def split_text(docs: list[Document]):
def split_text(docs: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200, add_start_index=True) 
    split_docs = text_splitter.split_documents(docs)
    return split_docs

split_docs = split_text(docs)
print(len(split_docs))
print(split_docs[1].page_content[:3000])



32
Developer skills

Developer skillsResources for developers to grow in their skills and careers.

Application developmentInsights and best practices for building apps.
Career growthTips & tricks to grow as a professional developer.
GitHubImprove how you use GitHub at work.
GitHub EducationLearn how to move into your first professional role.
Programming languages & frameworksStay current on what’s new (or new again).


Get started with GitHub documentationLearn how to start building, shipping, and maintaining software with GitHub.Learn more


Engineering

EngineeringGet an inside look at how we’re building the home for all developers.


In [110]:
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from pydantic_core.core_schema import model_field
def embedding_model(model_name: str, split_docs):
    texts = [doc.page_content for doc in split_docs]
    model_for_embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    vectors = model_for_embedding.embed_documents(texts)
    return vectors, model_for_embedding

embeddings, model_for_embedding = embedding_model(model_name="sentence-transformers/all-mpnet-base-v2", split_docs=split_docs)



In [111]:
# Quick check of embedding dimensions for one chunk
sample_embedding = model_for_embedding.embed_query(split_docs[0].page_content)
print(len(sample_embedding))

# print(len(embeddings))

768


In [118]:
from langchain_chroma import Chroma

def create_vector_db(text_docs: list[Document], embeddings_model_obj):
    return Chroma(
        collection_name="agent-collection",
        embedding_function=embeddings_model_obj,  # embedding function , called here and while creation of vector store the embeddings are calculated
        persist_directory="chroma_db"
    )

vector_store = create_vector_db(split_docs, model_for_embedding)
 
## the vector store will store text, metadata and embeddings
doc_ids = vector_store.add_documents(split_docs)


In [123]:
record = vector_store.get(ids=[doc_ids[0]], include=["embeddings", "documents", "metadatas"])
print("Text:", record["documents"][0][:300])
print("Embedding dim:", len(record["embeddings"][0]))
print("First 5 dims:", record["embeddings"][0][:5])


Text: How to write a great agents.md: Lessons from over 2,500 repositories - The GitHub Blog































































				Skip to content			

					Skip to sidebar				






/

	Blog

Changelog
Docs
Customer stories

















			Try GitHub Copilot		





			See 
Embedding dim: 768
First 5 dims: [ 0.01634688  0.03520707 -0.04967269 -0.02839436 -0.03164455]



### RAG agents
One formulation of a RAG application is as a simple agent with a tool that retrieves information. We can assemble a minimal RAG agent by implementing a tool that wraps our vector store:

